In [15]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [16]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [17]:
print(tf.__version__)

2.11.0


In [18]:
tf.random.set_seed(42)
np.random.seed(42)

In [19]:
df = pd.read_csv('BTC-Indicators.csv')
df=df.drop('Unnamed: 0',axis=1)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
0,2014/10/28,353.214996,359.984009,352.678986,357.618011,357.618011,7845880.0,358.928859,373.135500,360.189252,365.640480,26.597270,0.000000,407.000618,341.261085,-4.591695
1,2014/10/29,357.088989,357.833008,335.342987,335.591003,335.591003,18192700.0,352.133575,368.908214,354.038225,361.569705,22.686407,0.000000,409.625952,335.692252,-6.585445
2,2014/10/30,335.709015,350.912994,335.071991,345.304993,345.304993,30177900.0,350.260433,366.247428,351.854527,359.371035,32.401375,0.170521,410.516219,333.176283,-7.297547
3,2014/10/31,345.009003,348.045013,337.141998,338.321014,338.321014,12545400.0,347.399863,363.001929,348.470696,356.530731,29.646025,0.122158,411.960112,329.334590,-8.329426
4,2014/11/1,338.649994,340.528992,321.054993,325.748993,325.748993,16677200.0,344.325291,358.309572,342.789699,352.383942,21.806794,0.000000,413.709058,322.305642,-10.045852


In [20]:
df.shape

(3046, 16)

In [21]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,3046.0,1.319543e+04,1.612375e+04,1.768970e+02,7.518445e+02,7.361770e+03,1.835398e+04,6.754973e+04
High,3046.0,1.352535e+04,1.653489e+04,2.117310e+02,7.644685e+02,7.528680e+03,1.884302e+04,6.878962e+04
Low,3046.0,1.282973e+04,1.564943e+04,1.715100e+02,7.390708e+02,7.224057e+03,1.773179e+04,6.638206e+04
Close,3046.0,1.320146e+04,1.611962e+04,1.781030e+02,7.527688e+02,7.364940e+03,1.836853e+04,6.756683e+04
Adj Close,3046.0,1.320146e+04,1.611962e+04,1.781030e+02,7.527688e+02,7.364940e+03,1.836853e+04,6.756683e+04
Volume,3046.0,1.671192e+10,1.987820e+10,6.491650e+06,1.268932e+08,9.037149e+09,2.821866e+10,3.509680e+11
SMA_7,3046.0,1.317872e+04,1.609929e+04,2.045456e+02,7.411929e+02,7.330519e+03,1.819392e+04,6.551088e+04
SMA_14,3046.0,1.315154e+04,1.607668e+04,2.228731e+02,7.377346e+02,7.321023e+03,1.805389e+04,6.398306e+04
EMA_7,3046.0,1.317859e+04,1.608719e+04,2.158651e+02,7.399243e+02,7.357175e+03,1.808245e+04,6.473356e+04
EMA_14,3046.0,1.315193e+04,1.605150e+04,2.252705e+02,7.343472e+02,7.358198e+03,1.788575e+04,6.394564e+04


In [22]:
#將時間轉換為數值，以方便後續可以運算
timestamp_s = pd.to_datetime(df['Date']).map(datetime.datetime.timestamp)
df['Coin_timestamp'] = timestamp_s

In [23]:
df = df.drop(['Date'], axis=1)
df.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD,Coin_timestamp
0,353.214996,359.984009,352.678986,357.618011,357.618011,7845880.0,358.928859,373.135500,360.189252,365.640480,26.597270,0.000000,407.000618,341.261085,-4.591695,1.414426e+09
1,357.088989,357.833008,335.342987,335.591003,335.591003,18192700.0,352.133575,368.908214,354.038225,361.569705,22.686407,0.000000,409.625952,335.692252,-6.585445,1.414512e+09
2,335.709015,350.912994,335.071991,345.304993,345.304993,30177900.0,350.260433,366.247428,351.854527,359.371035,32.401375,0.170521,410.516219,333.176283,-7.297547,1.414598e+09
3,345.009003,348.045013,337.141998,338.321014,338.321014,12545400.0,347.399863,363.001929,348.470696,356.530731,29.646025,0.122158,411.960112,329.334590,-8.329426,1.414685e+09
4,338.649994,340.528992,321.054993,325.748993,325.748993,16677200.0,344.325291,358.309572,342.789699,352.383942,21.806794,0.000000,413.709058,322.305642,-10.045852,1.414771e+09


In [24]:
#二值化收盤價，昨天比今天高就是跌，就為False
# UpDown=[True]
# for i in range(1,len(df['Close'])):
#     if df['Close'][i]>df['Close'][i-1]:
#         UpDown.append(True)
#     else:
#         UpDown.append(False)
# df['UpDown'] = UpDown

In [25]:
#將資料集分為70%訓練、20%驗證、10%測試
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((2132, 16), (609, 16), (305, 16))

In [26]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [27]:
train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,2132.0,0.219520,0.212924,0.0,0.013850,0.180106,0.401374,1.0
High,2132.0,0.217044,0.212814,0.0,0.012002,0.176346,0.398726,1.0
Low,2132.0,0.219254,0.211140,0.0,0.014215,0.183112,0.400347,1.0
Close,2132.0,0.219493,0.212802,0.0,0.013792,0.180044,0.400975,1.0
Adj Close,2132.0,0.219493,0.212802,0.0,0.013792,0.180044,0.400975,1.0
Volume,2132.0,0.106368,0.156193,0.0,0.000746,0.028229,0.174537,1.0
SMA_7,2132.0,0.235121,0.229267,0.0,0.013405,0.193733,0.432332,1.0
SMA_14,2132.0,0.243595,0.238488,0.0,0.012650,0.203215,0.446403,1.0
EMA_7,2132.0,0.234503,0.228941,0.0,0.012773,0.194779,0.429802,1.0
EMA_14,2132.0,0.252283,0.246344,0.0,0.012853,0.212647,0.458585,1.0


In [29]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')